![](https://www.soyhenry.com/_next/static/media/HenryLogo.bb57fd6f.svg)

# Introducción a las bases de datos vectoriales 
## Clase #2 :Transformers


In [ ]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL_NAME = "gpt-4o-mini"

In [ ]:
text = "Nunca es demasiado tarde para ser sabio."
encoding = tiktoken.encoding_for_model(MODEL_NAME)
# Convertir texto a tokens (IDs)
token_ids = encoding.encode(text)
print(f"IDs de Tokens: {token_ids}")

# Ver los tokens en texto (decodificando uno por uno para inspeccionar)
tokens_text = [encoding.decode_single_token_bytes(token).decode('utf-8') for token in token_ids]
print(f"Tokens de texto: {tokens_text}")

# Mostrar tabla comparativa (visualización típica del Cap 3)
df_tokens = pd.DataFrame({"Token ID": token_ids, "Token Text": tokens_text})
print(df_tokens)

In [ ]:
response = client.embeddings.create(
    input=text,
    model="text-embedding-3-small"
)

embedding_vector = response.data[0].embedding
print(f"Dimensión del vector: {len(embedding_vector)}") 
print(f"Primeros 10 valores: {embedding_vector[:10]}")

In [ ]:
prompt = "La capital de Argentina es "

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=15,
    logprobs=True,      # <--- Esto habilita la inspección
    top_logprobs=5      # <--- Ver las top 5 opciones para cada token
)

# Extraer el contenido generado
generated_content = response.choices[0].message.content
print(f"Prompt: {prompt}")
print(f"Completado: {generated_content}\n")

In [ ]:
print("--- Inspección de Probabilidades (Looking Inside) ---")
for i, token_data in enumerate(response.choices[0].logprobs.content):
    token = token_data.token
    prob = np.exp(token_data.logprob) * 100  # Convertir logprob a % real
    
    print(f"\nPaso {i+1}: Token elegido -> '{token}' ({prob:.2f}% confianza)")
    
    print("   Otras opciones consideradas:")
    for top_k in token_data.top_logprobs:
        top_prob = np.exp(top_k.logprob) * 100
        print(f"     - '{top_k.token}': {top_prob:.2f}%")